# Notes: Hierarchical Models

In which we will
* Imagine devising models for more complex situations than we've seen so far.
* Introduce ourselves to the use of nuisance parameters and hierarchical structure in models.

Now that you've gained some experience carrying our Bayesian inference, it's time to face the fact that real life is often _much_ more complicated than the problems we've worked so far. (The original title of these notes, years ago, was "Coping with Reality".)

In any case, real data sets tend to be complicated. Some common features:
* Heteroscedastic (different for each data point, whatever that means) measurement uncertainties
* Measurement uncertainty for *every* measured quantity
* Correlated measurement uncertainties, amongst different quantities and/or data points
* Intrinsic scatter (i.e. real differences that are not just due to measurement error)
* Selection effects and systematially incomplete data
* Presence of unwanted sources in the data set
* Systematic uncertainties every which way

All of these things represent something that the Universe/atmosphere/telescope/etc. has done to the signal we measure before we get our hands on it, carve it in stone, and call it data. (Insert the adjective of your choice before "Universe", if you'd like.) Recalling our notes on [generative models](generative_models.ipynb), that means that any of these effects that apply deserve to be represented in our model. Very often, this involves introducing additional *nuisance parameters* and/or expanding the model such that it has a *hierarchical* structure.

## Nuisance parameters

We've already seen nuisance parameters; they're any model parameter that we are not particularly interested in, at the end of the day. This is an entirely subjective label - nuisance parameters are not formally or functionally treated any differently from other parameters. We come back to them now because adding nuisance parameters to a model provides a way to *explicitly account for and marginalize over systematic uncertainties*. This can be as simple as assigning a prior distribution to some quantity that would otherwise remain fixed, or it can involve a more explicit expansion of the model. Nuisance parameters often represent **latent variables** - things that are logically or physically part of a model, but which cannot be directly measured.

### Example: measuring flux with a background

Recall our simple measurement of a source's flux based on a number of detected counts from way back in the [Bayes' Law notes](bayes_law.ipynb). Let's explicitly include the constant conversion between flux and counts, so that our model might be described by
* $F \sim \mathrm{Gamma}(\alpha,\beta)$
* $\mu = C\,F$
* $N \sim \mathrm{Poisson}(\mu)$

The form of the prior on $F$ isn't critical for our purposes here, so we'll stick with the Gamma distribution. The corresponding PGM looks like this:

<table>
    <tr>
        <td><img src="graphics/hier_poissoneg_pgm1.png" width=70%></td>
    </tr>
</table>

Now let's expand the model to account for the fact that our measurement actually includes counts from both the source (a galaxy, say) and some background. Being good astronomers, we also remembered to take a second measurement of a source-free (background-only) patch of sky.

With subscripts $b$ for background, $g$ for galaxy, $s$ for the science observation that includes both, our model might be
* $F_b \sim \mathrm{Gamma}(\alpha_b,\beta_b)$
* $\mu_b = C_b\,F_b$
* $N_b \sim \mathrm{Poisson}(\mu_b)$
* $F_g \sim \mathrm{Gamma}(\alpha_g,\beta_g)$
* $\mu_s = C_s(F_g+F_b)$
* $N_s \sim \mathrm{Poisson}(\mu_s)$

<table>
    <tr>
        <td><img src="graphics/hier_poissoneg_pgm2.png" width=70%></td>
    </tr>
</table>

We'd probably regard $F_b$ and $\mu_b$ as nuisance parameters here - we need to account for our uncertainty in $F_b$, but measuring it isn't the point of the analysis.

More interestingly, suppose we want to marginalize over systematic uncertainty in the instrument gain, assuming it's the same for both observations. Maybe the instrument team told us it was known to $\sim5\%$. We might introduce a _new_ parameter, $\gamma$, representing the true (unknown) value of the gain relative to its published value (which would be included in $C$ already). Observe how the model changes:
* $\gamma \sim \mathrm{Normal}(1.0, 0.05)$
* $F_b \sim \mathrm{Gamma}(\alpha_b,\beta_b)$
* $\mu_b = \gamma C_b\,F_b$
* $N_b \sim \mathrm{Poisson}(\mu_b)$
* $F_g \sim \mathrm{Gamma}(\alpha_g,\beta_g)$
* $\mu_s = \gamma C_s(F_g+F_b)$
* $N_s \sim \mathrm{Poisson}(\mu_s)$

<table>
    <tr>
        <td><img src="graphics/hier_poissoneg_pgm2a.png" width=70%></td>
    </tr>
</table>

## Hierarchical models

Often, the model for a data set takes a hierarchical structure. This is naturally the case when we are measuring properties of multiple sources in order to learn about properties of a _class_ of sources to which they belong. It's honestly difficult to think of astrophysical use cases where this doesn't apply, in principle.

In statistics, the applicability of hierarchical models is related to the concept of **exchangeability** - meaning that, as far as we know, individual sources of a given class are equivalent (until we measure them). In other words, exchangeability means that it's fair to apply a common prior distribution to properties of multiple subjects (be they astronomical sources, people, etc.). This prior parameters describe the statistical properties of the source class, and are often what we're most interested in measuring.

To be mind-numblingly formal for a moment, the general form for a hierarchical model goes like this:
* $p(\mathrm{data}|\theta)$ describes the measurement process (what we've been calling the sampling distribution),
* $p(\theta)$ decomposes as $p(\theta|\phi_1)\,p(\phi_1)$,
* $p(\phi_1)$ decomposes as $p(\phi_1|\phi_2)\,p(\phi_2)$,
* $\ldots$,
* $p(\phi_n)$, usually taken to be "uninformative"

In the simplest hierarchical case, $\theta$ would represent the properties of individual sources that we've observed; $\phi_1$ would parametrize the statistical properties of the class of sources, $p(\theta|\phi_1)$; and $\phi_2$ would be the hyperparameters of some prior distribution for $\phi_1$, $p(\phi_1|\phi_2)$. Note that there is now some ambiguity as to what we call "the priors" of the model - $p(\theta|\phi_1)$ is _the prior on $\theta$_, but it is also an integral part of the model, not representing prior knowledge as such. You might recall earlier comments that it's best to regard the choice of priors as part of the model definition; this is one reason for it.

### Example: galaxy luminosity function

Let's modify the previous example as follows
* We're interested in luminosity rather than flux - if we know the distance to the target, this just means including another known factor in $C_g$, which now converts counts to $L$.
* We'll measure $m>1$ galaxies, and are interested in constraining the luminosity function, traditionally modelled as

$n(x) = \phi^\ast x^\alpha e^{-x}; \quad x=\frac{L}{L^\ast}$

where $n$ is the number density of galaxies.

<table>
    <tr>
        <td><img src="graphics/hier_lumfcn.png" width=70%></td>
    </tr>
</table>

For simplicity, we'll assume that we've measured *every* galaxy above a given luminosity in some volume. This is not very realistic, but the issues raised by incomplete data sets later on. For simplicity, let's also assume that the same background applies to each galaxy measurement, and that we have a galaxy-free observation of it, as before.

Compressing the $L\rightarrow N$ and $F\rightarrow N$ conversions, the PGM for our experiment looks like this:

<table>
    <tr>
        <td><img src="graphics/hier_poissoneg_pgm3.png" width=70%></td>
    </tr>
</table>

### Example: linear regression with most of the trimmings

Consider the following in the context of fitting a line to $x$ and $y$ data.

* Heteroscedastic (different for each data point, whatever that means) measurement uncertainties
* Measurement uncertainty for *every* measured quantity
* Correlated measurement uncertainties, amongst different quantities and/or data points
* Intrinsic scatter (i.e. real differences that are not just due to measurement error)

In the generative sense, this means:
* The Universe has some distribution of the $x$ values of things. Call the true (unknowable) values of $x$ for all the things $\xi$.
* For a given $\xi$, there is some variance in the true $y$ values of things (call these $\eta$). Their mean value as a function of $\xi$ follows a line, parametrized by $(\alpha,\beta)$, with the shape of the PDF given by $\Sigma$.
* We don't know $\xi$ or $\eta$ for any source, but we make measurement of them and call the associated data $x$ and $y$. The measurement uncertainties are corrlated, meaning that $(x,y)$ is drawn from some 2D PDF that depends on the true values $(\xi,\eta)$ and some other parameters, $M$, that we will mercifully assume we know.

Stare at this long enough and you will see the classic least-squares line fitting scenario in there somewhere - but with a lot more fun added in. Here is a PGM based on those words:

<table>
    <tr>
        <td><img src="graphics/regression_pgm.png" width=90%></td>
    </tr>
</table>

Note that we are still assuming that our set of measured things is not systematically incomplete somehow. We'll deal with that noise later in the course.

### Example: class membership

What if our set of observed sources contains members of multiple classes? We could imagine adding a discrete "group label" parameter, $g$, to the model for each source. The prior for $g$ would be multinomial, depending on the (possibly unknown) fractions of sources belonging to each class,

$p(g_i|\{f_j\}) = \mathrm{Multinomial}(\{f_j\})$,

with $i$ indexing over sources and $j$ over classes. $g_i$ then determines which class model provides the sampling distribution for the measurements of the $ith$ source.

Alternatively, we could just implement the sampling distribution as a sum over possible classes, which is equivalent to marginalizing over $g_i$ analytically,

$p(\mathrm{data}_i|\theta_1, \theta_2, \ldots) = \sum_j f_j\,p_j(\mathrm{data}|\theta_j)$.

Here $p_j(\mathrm{data}|\theta_j)$ would be the appropriate sampling distribution for the $j$th class.